
<a href="https://colab.research.google.com/github/isikus/MachineScorning/blob/main/Нагенерировать%20машинного%20отупения%20на%20ruGPT3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Нагенерировать машинного отупения на ruGPT3

In [1]:
import os
import json

def is_empty(fpath, default_lines=[["Дополни эту фразу", "И эту фразу тоже дополни"], ["Первый пример запроса", "Из нескольких строк", "", "И второй пример", "Я тоже придумать смог"]]):
  if not os.path.isfile(fpath):
    return True
  if not (os.path.getsize(fpath) > 0):
    return True
  if default_lines:
    with open(fpath, "r") as infile:
      lines = infile.read().split("\n")
    for deflines in default_lines:
      if lines == deflines:
        return True
  return False
  

def get_lbc(model_meta):
  lbc = model_meta["line_break_character"]
  lbc = lbc.replace("\s", " ")
  return lbc

#@markdown Для начала выберите модель:

Модель = "Нейротексты «Короля и Шута»"  #@param ["Нейроописания товаров", "Нейроинструкции на все случаи жизни", "Нейротексты «Короля и Шута»", "Нейроновости Северной Кореи", "Нейропоздравления с новым годом"]

desc_dict = {
    "Нейроописания товаров": "products_desc",
    "Нейроинструкции на все случаи жизни": "wikihowto",
    "Нейротексты «Короля и Шута»": "kishverses",
    "Нейроновости Северной Кореи": "rukcna",
    "Нейропоздравления с новым годом": "greetings"
}

model = desc_dict[Модель]

!wget -q --show-progress https://storage.googleapis.com/ml-bucket-isikus/machine-scorning/{model}.tar.gz
!mkdir {model}
!tar -xzf {model}.tar.gz -C {model}

try:
  with open(model+"/meta.json", "r") as injson:
    model_meta = json.load(injson)
  print('\x1b[1m'+Модель + ": модель подготовлена"+'\x1b[0m')
except Exception as e:
  print('\x1b[1;31m'+"Что-то сломалось – модель не готова :("+'\x1b[0m')

kishverses.tar.gz   100%[===================>] 443.64M   101MB/s    in 4.9s    
Нейротексты «Короля и Шута»: модель подготовлена


Теперь запишите варианты начала сообщений, которые должна дополнить модель, сразу под текстом `%%writefile input.txt`, по 1 разному началу на строку:

In [2]:
%%writefile input.txt
Дополни эту фразу
И эту фразу тоже дополни

Writing input.txt


**Важно:** если ваши запросы содержат переносы строк (например, это начало стихотворения), вместо предыдущей клетки, заполните клетку ниже, разбивая ваш ввод двумя переносами строк. Код ниже заменит переносы строк на специальный символ – это необходимо для корректной работы модели.

In [3]:
%%writefile input_with_lines.txt
Первый пример запроса
Из нескольких строк

И второй пример
Я тоже придумать смог

Writing input_with_lines.txt


Если у вас совсем нет идей, запустите код ниже – он скачает примеры запросов по умолчанию для вашей модели:

In [ ]:
if is_empty("input.txt") and is_empty("input_with_lines.txt"):
  !cp {model}/example_input.txt input.txt
  print('\x1b[1;31m'+"Загружены примеры по умолчанию"+'\x1b[0m')

In [5]:
#@markdown Наконец, укажите, сколько вариантов выдачи нужно сгенерировать модели на каждый из запросов (от 1 до 30):

Повторов = 4  #@param {"type": "integer"}
takes = Повторов

if takes < 1:
  takes = 1
  print("Это маловато. Установили значение " + str(takes) + ".")
if takes > 30:
  takes = 30
  print("Это многовато. Установили значение " + str(takes) + ".")
if type(takes) != int:
  takes = 4
  print("Что-то пошло не так. Установили значение " + str(takes) + ".")

## Подготавливаем среду для модели
Сами файлы скачаны, но нужно ещё выполнить код, чтобы подготовить модель к работе.

Проверяем, что мы подключены к видеокарте: команда в клетке ниже должна работать и показывать показывать примерно 12000-15000 Мб свободной памяти:

In [8]:
!nvidia-smi

Sun Feb 14 12:55:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Устанавливаем нужные версии библиотек:

In [9]:
!pip3 install -q urllib3==1.25.4 transformers==2.8.0

print('\n'+'\x1b[1m'+"Требуемые версии пакетов установлены"+'\x1b[0m')

     |████████████████████████████████| 133kB 8.6MB/s 
     |████████████████████████████████| 573kB 17.1MB/s 
     |████████████████████████████████| 3.7MB 19.4MB/s 
     |████████████████████████████████| 890kB 58.9MB/s 
     |████████████████████████████████| 133kB 60.0MB/s 
     |████████████████████████████████| 1.2MB 48.7MB/s 
     |████████████████████████████████| 71kB 11.5MB/s 
     |████████████████████████████████| 7.2MB 57.1MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.

Требуемые версии пакетов установлены


Скачиваем файл с исходным кодом для работы модели:

In [10]:
!wget -q --show-progress https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

print('\n'+'\x1b[1m'+"Код запуска модели загружен"+'\x1b[0m')

generate_transforme 100%[===================>]  10.23K  --.-KB/s    in 0s      

Код запуска модели загружен


Запускаем код, необходимый для преобразования файлов и отображения выдачи:

In [11]:
import re
import os
import sys
import json
import random
from datetime import datetime

from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

with open(model+"/meta.json", "r") as injson:
  model_meta = json.load(injson)

# converting inputs

s = ""

if not is_empty("input_with_lines.txt"):
  with open("input_with_lines.txt", "r", encoding="utf-8") as intxt:
    prompts = [get_lbc(model_meta).join([sp for sp in p.split("\n") if sp]) for p in intxt.read().split("\n\n") if p]
else:
  with open("input.txt", "r", encoding="utf-8") as intxt:
    prompts = [p for p in intxt.read().split("\n") if p]

for prompt in prompts:
  s += (model_meta['prompt'] % prompt + '\n') * takes

with open("rawi.txt", "w", encoding="utf-8") as rawo:
  rawo.write(s)

print('\x1b[1m'+"Среда подготовлена"+'\x1b[0m')

Среда подготовлена


## Генерируем отупение

Теперь всё, наконец, готово к тому, чтобы модель сгенерировала нам выдачу. То, насколько серьёзный и осмысленный текст будет пытаться выдавать модель, можно регулировать по шкале **`Бредовость`**: здесь есть 5 степеней – от самой нормальной до самой странной (я предупреждал). Кроме этого, флажок **`Повторяться`** определяет, как будет работать модель: если галочку оставить, то первый результат будет очень похож не те, что есть в канале, а последующие будут с ним совпадать. А если снять галочку, то выдача модели каждый раз будет новая. Удачи!

In [12]:
#@markdown ### Начать генерацию!

Бредовость = 75 #@param {type: "slider", min: 0, max: 100, step: 25}
stlv = Бредовость

stup_cases = {
    0: {"k": 5, "p": 0.95, "temp": 1.0},
    25: {"k": 5, "p": 0.93, "temp": 2.0},
    50: {"k": 6, "p": 0.91, "temp": 3.0},
    75: {"k": 6, "p": 0.9, "temp": 4.0},
    100: {"k": 7, "p": 0.88, "temp": 5.0}
}

Повторяться = True   # @param {"type": "boolean"}
add_randomness = not Повторяться

rs = random.choice(list(range(0, 10000))) if add_randomness else 42
printmd("Ключ случайного значения: **"+str(rs)+"**")

responses = !python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path={model} \
    --k={str(stup_cases[stlv]["k"])} \
    --p={str(stup_cases[stlv]["p"])} \
    --seed={str(rs)} \
    --temperature={str(stup_cases[stlv]["temp"])} \
    --repetition_penalty=1.0 \
    --stop_token="{model_meta['sequence_end_character']}" \
    --length=500 < rawi.txt 2>/dev/null

responses = "\n".join(list(responses))

lbc = model_meta["line_break_character"]
ssc = model_meta["sequence_start_character"]
sec = model_meta["sequence_end_character"]
prompt = re.sub(r"%[a-z]+", "(.*?)", model_meta["prompt"])

resps = [r for r in responses.split("Context >>> ruGPT:\n") if r]
sps = [sp for sp in s.split("\n") if sp]
for sp, resp in zip(sps, resps):
  pt = re.search(re.compile(repr(re.sub(r"%[a-z]+", "(.*)", model_meta["prompt"]))[1:-1]), sp).group(1)
  rs, _ = re.findall(re.compile(re.escape(sp) + r"\n?(.*?)(" + ssc + "|$)", flags=re.DOTALL), resp)[0]
  pt = "<br>".join("**" + p + "**" for p in re.split(lbc, pt) if p)
  printmd(pt)
  rs = re.sub(lbc, "<br>", rs).strip()
  rs = re.sub(r"([^\s])<br>([^\s])", r"\1 <br> \2", rs)
  printmd(rs[:rs.find("Context >>")] + "<br>")

Ключ случайного значения: **42**

**Я начал жизнь в трущобах городских**<br>**И добрых слов я не слыхал.**<br>**Когда ласкали вы детей своих,**<br>**Я есть просил, я замерзал.**

// И я вас любил. И в ответ мне вы говорили - <br> "Нет, не стоит! Ты не можешь жить со мною в трущобах! <br> Я не могу, я умирающий, умереть хочу!" - (Пауза, потом он говорит: "Погибли бы, не узнав меня ты! ")" (Припев). <br>Я жил и жил в сквере, <br> А в кустах, под сосною <br> В тот момент он увидел <br> Клоуна, он упал и замер, - <br>"Опомнись парень!" <br> Припев<br>

**Я начал жизнь в трущобах городских**<br>**И добрых слов я не слыхал.**<br>**Когда ласкали вы детей своих,**<br>**Я есть просил, я замерзал.**

<br>И я вас спас! <br> Но вы, конечно же - ведьма, <br> Меня обманули.  </s  <p style="pic": none;  margin: auto;  min-wheelpas: 15px;  position: absolute; left: auto <br> Все права на картинки и тексты принадлежат their respective directive -  http://www.sovet-club-ru.com/viewtopic.php?id=5706; /* Font-weard:s VCL-4 (Бельгийское кино, 2009, Режисеры - Александр Гольденберг  <div align*=right"&gt;ПРИЯТИЯ&hearted<br>

...

, я в аду. <br> Но, клянусь, в этом не будет ничего плохого. <br> И, когда солнце поднимется, 

Скажу тебе: <br> Я умую руки <br> В подвале, под душем <br> С водой. 
Пей с молоком и ешь со сметаной. <br> И в этом мире ты не умрешь! <br>Я не буду с тобою жить - <br> Не будет у меня любви, поверь. <br> Но я верю - ты верь. <br> Я буду ждать тебя вечно. <br> Ты вернешься со своей водой! 
  (finished version of "Прогулка по кладбищу"). 
   Стих: Я, парень, в этом городе был <br> Я, паря, как ангел. 

Я не был в раю, я не пил, я не ел - <br> Я, как ангел. <br> Но я все больше верил в чудеса. <br> Я, паря - не ангел! (Припев) 
   И, не смотря на свой возраст - <br> Верить, я все больше и чаще. <br> И я все дальше уезжался. <br>Я в этой глуши бывал, но, как-то, вдруг <br> Меня вдруг куда-то уплыли.
(Февраль, 2005, ср.: http://video.yadn.com/...✂ );"Не помню ни единого случая в истории моего пребывания во Франции, где я был. <br> Я бывал во всем. Я всегда бывал в этом. В каждом из них я узнавал что-то новоселье или день рождения, или же, наоборот: день свадьбы или дня смерти - это было всегда, а теперь я не понимаю - почему меня не приглашают в Париж? Я же здесь живу, у себя в городе! " <br> (finished version of «Прочь из города, от сюда!  <font face: left, color: black, size-size: 20pt;">)
   Стих: Мне, парнишка в этот миг
Не знаю, как тебе - мне
Прикаляться <br> Над всем подряд:
- Эй, парень! <br> Ты не понимаешь.
Эй, пацану лет пять-шесть!  </font>
   Стихо:  В этом лесу <br> Я бродил с утра и вечер - <br> Мне очень одиноко было в этот день..<br>

**Я умываю руки**<br>**Рука руку пачкает**<br>**Всюду зараза**<br>**Я никому не верю**

!/ Я умою руки, <br> Кровь моя - я, <br> Ты умой, ты, ты <br> Руки, я тебя. <br> Ты - моя душа.

Я хочу, я верю <br> Что ты умоеш меня! <br> Что ты скажешь, если б я был <br> Твой принц!
"Выйдем, выйдем" - 
Припев,  -  и мы вышли!
Мы пришли, мы пошли...
И, как ни странно  <Михалыч>,  я  все же не стал  плакать,  -  и он не побежал, не закричал...
Я  помню, как в первый вечер в подвале я нашел дневник, где было все: и слова, которые  мне шепчут мои родители, и дневник мой в углу. <br> "Я хочу тебя, ты мне померещился! <br> Хочу с тобой жить!" - говорил он, и все повторял я: <br> "Я хочу тебя, а не тебя! Я хочу умереть в твоем замке!" 
Я был так ошарашено потрясен тем ужасом,  который я пережил  и в какой момент  я его потерял! Я не сразу узнал  этот крик,  и все же я  не стал ему сопротивляться! Но, как я и сказал - "Я очень долго ждал, когда  твой принц умрет, я не верил в него!" И,  как ни старался, не смог этого понять!
Припев,
-  -  И ты хочешь умереть? <br> Ты очень хочешь умереть, мой мальчик? <br> А почему я ничего о тебе не знаю?  - "Прикольно, что он умер! А ты, что же ты, мне его не рассказал? Ведь я все время о тебе думал и даже ночью в подвал заходил".
- "Да, милый мой,  это я все тебе расскажу,  когда мы с тобой  увиженнем!" – прошептал мой бедный принц, но, когда дверь подвала закрылась за его телом и он исчез во мгле ночи, мне стало совсем жутко, как в подвале!
- "Что за чертовщину мне сниться все это время, милый мой? Ты ведь не знаешь, кто я, и почему ты меня убил!" - сказал мне  мой  мучительный демон.
Припев  -
"Что за чертовщину мне сниться это всё все это время... Я, наверное очень страшный страшный злодей... А ты кто? Я не могу с тобой ничего сделат<br>

**Я умываю руки**<br>**Рука руку пачкает**<br>**Всюду зараза**<br>**Я никому не верю**

, но мне <br> В душу не веришь, а хочешь <br> Сходи в баню! <br> Ты же, как я погляжу: <br> Ты все-таки человек <br> И, если честно признаться честно: <br> Ты мне не враг. <br> И если я тебя обманул, ты меня обманул! <br> Я, признаться тебе хочу, <br> Я, признаваясь в том не виню. <br> Я верю, верю <br> Ты не дурак <br> В меня, ты не веришь <br> Я, признаясь, в тебя не верю. <br> (Припев). <br> Я, признаваясь тебе хочу, <br>Я признаваясь в том не виню, ты не дурак <br>В меня, ты ни черта не понимаешь <br> И в душу не веришь. <br> Я признаваясь вам, признавшись, <br> Что в вас я был не прав <br> И не прав. <br> (Выхватывая из воздуха нож, парень стучал им в окно. Затем, схватив с полки нож с вилками, парень вышел из квартиры)  
<br>

## Больше отупения!

Хотите попробовать другие фразы? Тогда впишите их в клетку под этим текстом, как до этого, и запустите код ниже. Не забудьте поменять `%%writefile input.txt` на `%%writefile input_with_lines.txt`, если ваши запросы содержат переносы строк!

In [ ]:
%%writefile input.txt


Overwriting input.txt


In [13]:
#@markdown ### Генерируем новую выдачу

Повторов = 4  #@param {"type": "integer"}
takes = Повторов

if takes < 1:
  takes = 1
  print("Маловато повторов. Установлено значение " + str(takes) + ".")
if takes > 30:
  takes = 30
  print("Маловато повторов. Установлено значение " + str(takes) + ".")
if type(takes) != int:
  takes = 4
  print("Что-то пошло не так с количеством повторов. Установлено значение " + str(takes) + ".")

s = ""

if not is_empty("input_with_lines.txt"):
  with open("input_with_lines.txt", "r", encoding="utf-8") as intxt:
    prompts = [get_lbc(model_meta).join([sp for sp in p.split("\n") if sp]) for p in intxt.read().split("\n\n") if p]
else:
  with open("input.txt", "r", encoding="utf-8") as intxt:
    prompts = [p for p in intxt.read().split("\n") if p]

for prompt in prompts:
  s += (model_meta['prompt'] % prompt + '\n') * takes

with open("rawi.txt", "w", encoding="utf-8") as rawo:
  rawo.write(s)

Бредовость = 25 #@param {type: "slider", min: 0, max: 100, step: 25}
stlv = Бредовость

Повторяться = False   # @param {"type": "boolean"}
add_randomness = not Повторяться

rs = random.choice(list(range(0, 10000))) if add_randomness else 42
printmd("Ключ случайного значения: **"+str(rs)+"**")

responses = !python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path={model} \
    --k={str(stup_cases[stlv]["k"])} \
    --p={str(stup_cases[stlv]["p"])} \
    --seed={str(rs)} \
    --temperature={str(stup_cases[stlv]["temp"])} \
    --repetition_penalty=1.0 \
    --stop_token="{model_meta['sequence_end_character']}" \
    --length=500 < rawi.txt 2>/dev/null

responses = "\n".join(list(responses))

lbc = model_meta["line_break_character"]
ssc = model_meta["sequence_start_character"]
sec = model_meta["sequence_end_character"]
prompt = re.sub(r"%[a-z]+", "(.*?)", model_meta["prompt"])

resps = [r for r in responses.split("Context >>> ruGPT:\n") if r]
sps = [sp for sp in s.split("\n") if sp]
for sp, resp in zip(sps, resps):
  pt = re.search(re.compile(repr(re.sub(r"%[a-z]+", "(.*)", model_meta["prompt"]))[1:-1]), sp).group(1)
  rs, _ = re.findall(re.compile(re.escape(sp) + r"\n?(.*?)(" + ssc + "|$)", flags=re.DOTALL), resp)[0]
  pt = "<br>".join("**" + p + "**" for p in re.split(lbc, pt) if p)
  printmd(pt)
  rs = re.sub(lbc, "<br>", rs).strip()
  rs = re.sub(r"([^\s])<br>([^\s])", r"\1 <br> \2", rs)
  printmd(rs[:rs.find("Context >>")] + "<br>")

Ключ случайного значения: **267**

**Я начал жизнь в трущобах городских**<br>**И добрых слов я не слыхал.**<br>**Когда ласкали вы детей своих,**<br>**Я есть просил, я замерзал.**

<br>А в подвале я прятался, <br> Но однажды, очнувшись в темноте, <br> Я понял: <br> Я все, что было нужно, <br> В этом подвале украл. <br> (Припев) <br> Я в подвале прятался <br> В подвале, я дрожал, <br> И в подвале, я дрожал, <br> Но вдруг кто-то мне сказал: "Ты должен уйти. <br> Я не уйду!" <br> Я ушел из этой комнаты, <br> Но вдруг раздался жуткий вой. <br> (Припев) <br> И в тот же миг дверь подвала распахнулась: <br>Из темноты вышел страшный монстр <br> И на полу лежал скелет мой<br>

**Я начал жизнь в трущобах городских**<br>**И добрых слов я не слыхал.**<br>**Когда ласкали вы детей своих,**<br>**Я есть просил, я замерзал.**

<br>Я есть был в вас, и вы тоже, <br> И вы все это любили. <br> (Припев) <br> И вот однажды <br> На улице мне повстречался парень. <br> Припев: <br> "Ты, милый, не злись! <br> Ты мне не нравишься! <br> Я тебя очень сильно люблю! <br> Я тебя очень сильно люблю, <br> Я с тобой в одной крови! <br> Я с тобою в одном сердце, <br> Я с тобой во всем виноват! <br> И вот однажды, когда я спал, <br> Я вдруг увидел тебя. <br> Ты, наверно, тоже очень милый!  (Припев<br>

...

! <br> И я, как последний дурак <br> В этом мире останусь один, <br> А если что - буду жить<br>

**Я умываю руки**<br>**Рука руку пачкает**<br>**Всюду зараза**<br>**Я никому не верю**

, <br> А в сердце мне всегда тепло. <br> Я никому не верю <br> Я никому не верю <br> Что это не бред! <br> Я никому не верю <br>Что я никому не верю. <br> Я никому не верю <br> Что я никому не верю! <br> И в эту же секунду <br> Упал я с кровати <br> И на полу я валяюсь! <br>Я никому ни в чем не верю <br> Что это НЕ бред <br> И НЕ ТАНЦУЕТ!!! <br> Упал я с кровати <br> И на полу я валяюсь!  \t(C) А.С.ПОДГОЛОС.

"Я никому не верю, <br> А в сердцах мне всегда тепло, <br> Я никому не верю, <br> Я никому не верю, <br> Но я никому ничего не обещаю..."  /Припев  (с) А.С.ПОДГОЛОС. <br> / Припев: <br> Мне очень хорошо с тобою, <br> Очень плохо, <br> Что ты мне нравишься! <br> Я никому не верю <br> Я никому не верю, <br> Я никому ничего не обещаю... (Припев) <br>И в ту же секунду <br> Упал я с кровати, <br>И в тот же миг - <br>На полу я валяюсь! - <br> В комнате стало темно, и мне страшно!  \t(C) А.П.Чехов.

"Я никому не верю, <br> А в сердцах мне всегда тепло, <br> Я никому ничего не обещаю, <br> Я никому ничего не обещаю, <br> Но я никому ничего не обещаю. <br> (Припев) <br> И в ту же секунду <br> Упал я с кровати <br> И на полу я валяюсь. <br> (Припев) <br>И в ту же минуту <br> Упал я с кровати <br> И на полу я валяюсь... (Припев) <br>Я никому ничего не обещаю, <br> Я никому ничего не обещаю... (Припев) <br>Но я никому ничего не обещаю..." /Припев  (с) А.П.Чехов.<br<br>

**Я умываю руки**<br>**Рука руку пачкает**<br>**Всюду зараза**<br>**Я никому не верю**

<br>Я не верю <br> Я никому не доверяю, <br> Я никому ничего не доверяю <br> Я никому ни чего не доверяю, <br> Мне никто никогда не доверял, <br>Мне никому никто не доверял, <br> Мне никогда ничего не доверяла... " <br> И в тот миг, как я понял это, <br>Я был так же безумен, <br>Я не знал куда деться <br> И не мог я ничего с этим поделать <br>Я не мог ничего с этим поделать <br>Я ни черта ни чего не делал, <br> Я ни черта не делал, <br> Но мне все казалось не таким, <br> Я ни черта ничего не делал <br> Я просто не понимал что такое <br> Я ни чего не понимал, <br> Я ни черта ничего не делал, <br> Но вдруг я увидел в темноте свет, <br> И в ту же секунду я понял это, <br>В ту же секунду я понял что такое <br>Что значит "не знаю" я ничего не понимаю <br>Я просто ничего не делал, <br>Мне все кажется не таким, <br>И я ни чего ни делал, <br> Но внезапно мне в голову пришел гениальный план - <br> Я в подвале, в подвале, <br> А в подвале, в подвале <br> Я ничего не знаю! <br>Я не понимаю - что значит, <br> Но мне не страшно <br> В подвале <br> Я не понимаю <br> Что значит, <br> Но мне не страшно <br> В подвале <br> Я не понимаю <br> Что значит, <br> Что значит -  
<br>

На этом всё. Получайте удовольствие (надеюсь)!